In [26]:
import torch
import torch.nn as nn

class LR(nn.Module):
    def __init__(self):
        super().__init__()
        self.ll = nn.Linear(1, 1)
        self.loss_fn = nn.MSELoss()

    def forward(self, x, labels=None):
        preds = self.ll(x)

        if labels is not None:
            loss = self.loss_fn(preds, labels)
            return {"loss": loss, "logits": preds}

        return {"logits": preds}


In [27]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return {
            "x": self.x[idx],
            "labels": self.y[idx],
        }


In [37]:
inp = torch.tensor([[1.],[2.],[3.],[4.]])
out = torch.tensor([[1.],[2.],[3.],[4.]])

dataset = SimpleDataset(inp, out)

inp = torch.tensor([[5.],[6.],[7.],[8.]])
out = torch.tensor([[5.],[6.],[7.],[8.]])

eval_dataset = SimpleDataset(inp, out)


In [40]:
from transformers import TrainerCallback

class LRLossCallback(TrainerCallback):
    def __init__(self):
        self.history = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return

        if "loss" in logs:
            lr = logs.get("learning_rate")
            self.history.append({
                "step": state.global_step,
                "lr": lr,
                "loss": logs["loss"],
            })


In [65]:
from transformers import TrainingArguments

def make_args(lr, seed):
    return TrainingArguments(
        output_dir=f"./tmp/run_{exp}",
        per_device_train_batch_size=4,
        num_train_epochs=10,
        logging_steps=1,
        report_to="none",
        save_strategy="no",
        remove_unused_columns=False,  # ← IMPORTANT
        eval_strategy="epoch",
        fp16=False,
        learning_rate=lr,
        lr_scheduler_type="cosine",
        seed=seed
    )



In [66]:
all_loss_list = []


for exp in range(5):
    callback = LRLossCallback()
    lr_val = 10 ** (-exp)
    print("lr =", lr_val)

    model = LR()

    trainer = Trainer(
        model=model,
        args=make_args(lr_val, seed=exp),
        train_dataset=dataset,
        eval_dataset=eval_dataset,
        callbacks=[callback],
    )

    trainer.train()

    # extract logged losses
    losses = [
        log["loss"]
        for log in trainer.state.log_history
        if "loss" in log
    ]

    all_loss_list.append(losses)

    # history = callback.history
    # print(history)

Epoch,Training Loss,Validation Loss
1,5.076300,6.217173
2,2.051700,4.406288
3,1.501700,0.444595
4,0.032400,0.988045
5,0.123800,0.004100
6,0.022400,0.004147
7,0.012800,0.069165
8,0.006900,0.080134
9,0.010600,0.042920
10,0.005200,0.030780


Epoch,Training Loss,Validation Loss
1,9.423300,35.953953
2,7.304600,27.630583
3,5.498000,20.890980
4,4.052400,15.750909
5,2.965700,12.058399
6,2.197800,9.569510
7,1.689100,8.017708
8,1.377000,7.157342
9,1.206200,6.775292
10,1.131000,6.679074


Epoch,Training Loss,Validation Loss
1,0.221400,0.221167
2,0.189400,0.157624
3,0.160700,0.108485
4,0.136500,0.073108
5,0.117000,0.049394
6,0.102200,0.034596
7,0.091700,0.026051
8,0.084800,0.021607
9,0.080900,0.019714
10,0.079100,0.019246


Epoch,Training Loss,Validation Loss
1,19.895900,91.648773
2,19.863100,91.507187
3,19.831200,91.376022
4,19.801600,91.260971
5,19.775700,91.166161
6,19.754300,91.093781
7,19.738000,91.043777
8,19.726700,91.013962
9,19.720000,91.000153
10,19.716900,90.996613


Epoch,Training Loss,Validation Loss
1,5.850300,31.909046
2,5.848600,31.900692
3,5.846800,31.892946
4,5.845200,31.886147
5,5.843800,31.880545
6,5.842700,31.876263
7,5.841800,31.873306
8,5.841200,31.871542
9,5.840800,31.870724
10,5.840600,31.870514


In [61]:
import torch
print(torch.cuda.is_available())


False
